In [20]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import download_audio_gdrive
from util import split_audio_files
from util import get_time_span
from util import get_max_db_id


In [ ]:
df = read_spreadsheet(sheet_id = "1yKSzConuVWo8BuMDs2mabF5iiBKUz2wF--LIabFN6QE")

for index, row in df.iterrows():
    file_name = row['ID']
    gd_url = row['Audio link']
    Sr_no = row['z']
    
    # if not isinstance(gd_url, str) or not isinstance(id, str):
    #     continue
    if Sr_no >= 645 and Sr_no <= 691:
        gd_url = gd_url.split('?')[0]
        print(gd_url, file_name)
        # file_name = file_name + ".mp3"
        # download_audio_gdrive(gd_url, file_name)
        yt_downloaded = f"""yt-dlp --extract-audio --audio-quality 0 --audio-format wav --postprocessor-args "-ar 16000 -ac 1" {gd_url} -o './full_audio/{file_name}.%(ext)s'"""
        ! {yt_downloaded}



https://youtu.be/cXBHSXzuZY0 STT_AB00645
[youtube] Extracting URL: https://youtu.be/cXBHSXzuZY0
[youtube] cXBHSXzuZY0: Downloading webpage
[youtube] cXBHSXzuZY0: Downloading ios player API JSON
[youtube] cXBHSXzuZY0: Downloading android player API JSON
[youtube] cXBHSXzuZY0: Downloading m3u8 information
[info] cXBHSXzuZY0: Downloading 1 format(s): 251
[download] Destination: ./full_audio/STT_AB00645.webm
[download] 100% of    3.96MiB in 00:00:00 at 9.90MiB/s0;33m00:00
[ExtractAudio] Destination: ./full_audio/STT_AB00645.wav
Deleting original file ./full_audio/STT_AB00645.webm (pass -k to keep)
https://youtu.be/eQ5K41B88yo STT_AB00646
[youtube] Extracting URL: https://youtu.be/eQ5K41B88yo
[youtube] eQ5K41B88yo: Downloading webpage
[youtube] eQ5K41B88yo: Downloading ios player API JSON
[youtube] eQ5K41B88yo: Downloading android player API JSON
[youtube] eQ5K41B88yo: Downloading m3u8 information
[info] eQ5K41B88yo: Downloading 1 format(s): 251
[download] Destination: ./full_audio/STT_AB00

In [4]:
split_audio_files('STT_AB', 'wav')

['STT_AB00669.wav', 'STT_AB00658.wav', 'STT_AB00681.wav', 'STT_AB00684.wav', 'STT_AB00688.wav', 'STT_AB00679.wav', 'STT_AB00664.wav', 'STT_AB00655.wav', 'STT_AB00670.wav', 'STT_AB00676.wav', 'STT_AB00674.wav', 'STT_AB00666.wav', 'STT_AB00651.wav', 'STT_AB00660.wav', 'STT_AB00656.wav', 'STT_AB00662.wav', 'STT_AB00690.wav', 'STT_AB00671.wav', 'STT_AB00673.wav', 'STT_AB00665.wav', 'STT_AB00659.wav', 'STT_AB00654.wav', 'STT_AB00648.wav', 'STT_AB00677.wav', 'STT_AB00686.wav', 'STT_AB00645.wav', 'STT_AB00652.wav', 'STT_AB00672.wav', 'STT_AB00661.wav', 'STT_AB00663.wav', 'STT_AB00678.wav', 'STT_AB00646.wav', 'STT_AB00685.wav', 'STT_AB00668.wav', 'STT_AB00691.wav', 'STT_AB00667.wav', 'STT_AB00689.wav', 'STT_AB00650.wav', 'STT_AB00649.wav', 'STT_AB00680.wav', 'STT_AB00653.wav', 'STT_AB00675.wav', 'STT_AB00682.wav', 'STT_AB00657.wav', 'STT_AB00647.wav', 'STT_AB00687.wav', 'STT_AB00683.wav']
STT_AB00669.wav
./full_audio/STT_AB00669.wav STT_AB00669
1 9.18 vad
2 7.492500000000003 vad
3 5.9568749999

In [5]:
collect_segments('STT_AB', 'after_split', 'segments_ab')

Copied after_split/STT_AB00688/STT_AB00688_0015_293009_to_301193.wav to segments_ab/STT_AB00688_0015_293009_to_301193.wav
Copied after_split/STT_AB00688/STT_AB00688_0008_137843_to_146534.wav to segments_ab/STT_AB00688_0008_137843_to_146534.wav
Copied after_split/STT_AB00688/STT_AB00688_0005_105055_to_107434.wav to segments_ab/STT_AB00688_0005_105055_to_107434.wav
Copied after_split/STT_AB00688/STT_AB00688_0012_240409_to_243885.wav to segments_ab/STT_AB00688_0012_240409_to_243885.wav
Copied after_split/STT_AB00688/STT_AB00688_0001_11534_to_13896.wav to segments_ab/STT_AB00688_0001_11534_to_13896.wav
Copied after_split/STT_AB00688/STT_AB00688_0017_310144_to_312480.wav to segments_ab/STT_AB00688_0017_310144_to_312480.wav
Copied after_split/STT_AB00688/STT_AB00688_0020_415521_to_422457.wav to segments_ab/STT_AB00688_0020_415521_to_422457.wav
Copied after_split/STT_AB00688/STT_AB00688_0007_120276_to_122520.wav to segments_ab/STT_AB00688_0007_120276_to_122520.wav
Copied after_split/STT_AB006

In [6]:
! aws s3 cp segments_ab  s3://monlam.ai.stt/stt_pecha_tools/ --recursive

upload: segments_ab/STT_AB00645_0008_81852_to_84540.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0008_81852_to_84540.wav
upload: segments_ab/STT_AB00645_0002_32003_to_34500.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0002_32003_to_34500.wav
upload: segments_ab/STT_AB00645_0010_95166_to_97799.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0010_95166_to_97799.wav
upload: segments_ab/STT_AB00645_0001_3535_to_6623.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0001_3535_to_6623.wav
upload: segments_ab/STT_AB00645_0003_43900_to_48878.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0003_43900_to_48878.wav
upload: segments_ab/STT_AB00645_0005_49604_to_56985.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0005_49604_to_56985.wav
upload: segments_ab/STT_AB00645_0009_84924_to_94086.wav to s3://monlam.ai.stt/stt_pecha_tools/STT_AB00645_0009_84924_to_94086.wav
upload: segments_ab/STT_AB00645_0006_56985_to_64367.wav to s3://monlam.ai.stt/stt_pecha_tools/

In [2]:
target_directory = f"segments_ab"

In [5]:
from transformers import pipeline
from tqdm.auto import tqdm

from pathlib import Path

target_path = Path(target_directory)

rows = []
generator = pipeline(model="openpecha/wav2vec2_run8")

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file.name}", inf, get_time_span(str(file.name))])

  0%|          | 0/2779 [00:00<?, ?it/s]

In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])    

In [7]:
df.head()

,file_name,url,inference_transcript,audio_duration
0,STT_AB00651_0085_355598_to_358568,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ས་བོན་སྲིད་དར་ཐུབ་པ་རབ་ཏུ་སྐྱེ་བ་ཉིད་དུ་འགྱུར།,2.970
1,STT_AB00651_0054_218185_to_220328,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ལུང་གི་དོན་རིག་པས་བསྒྲུབ་པ་ལ་གཉིས།,2.143
2,STT_AB00651_0167_761205_to_763652,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,བཀག་པ་ལ་འཇིག་རྟེན་གྱིས་གནོད་པ་ལ་གནོད་བྱེད་བསྟན་པ།,2.447
3,STT_AB00658_0045_285815_to_292503,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ྱི་ཚོགས་ལ་བསམ་ཡུལ་ན་མེད་པའི་སྒྱུར་ལྡོག་གློ་བུར...,6.688
4,STT_AB00651_0804_3656213_to_3659402,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཁྱོད་ནི་སྐྱེ་བོ་དམ་པའི་གཞན་མི་འགྱུར་ཞིང་གང་གི་...,3.189


In [8]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

{'inference_transcript': {0: 'ས་བོན་སྲིད་དར་ཐུབ་པ་རབ་ཏུ་སྐྱེ་བ་ཉིད་དུ་འགྱུར།',
  1: 'ལུང་གི་དོན་རིག་པས་བསྒྲུབ་པ་ལ་གཉིས།',
  2: 'བཀག་པ་ལ་འཇིག་རྟེན་གྱིས་གནོད་པ་ལ་གནོད་བྱེད་བསྟན་པ།',
  3: 'ྱི་ཚོགས་ལ་བསམ་ཡུལ་ན་མེད་པའི་སྒྱུར་ལྡོག་གློ་བུར་བ་ཞིག་བྱུང་བ་དེའི་དབང་གིས་མང་ཚོགས་སྤྲུད། ཤེས་སྒྲ་དབང་ཕྱུག་རྒྱལ་པོ།',
  4: 'ཁྱོད་ནི་སྐྱེ་བོ་དམ་པའི་གཞན་མི་འགྱུར་ཞིང་གང་གི་ཕྱིར།',
  5: 'བྱང་སེམས་འཕགས་པའི་སའི་རྣམ་གཞག་བསྟན་པ་ལ་གསུམ།',
  6: 'སོ་སོ་སྲུང་ཐབས་གཙོ་དོན་བྱེད་པ་ལས། ཤེས་བྲའི་ཆི་སྲིམ་གྱི་ལས་གུས་དེ་ཁ་མྱོགས་ངེས་པ་ཞིབ་ཏུ་སྒྱུར་བར་དགག་བྱ་བཇི་རྒྱན་ཅིག་ཀྱང་བྱུང་མེད།',
  7: 'ཆོས་ལུགས་རྣམ་པ་མང་པོ་ཞིག་ཡོད་པའི་ནང་ནས་',
  8: 'དེ་བཞིན་མཁས་པ་རིན་པོ་ཆེ་ཡི་གླིང་དུ་ཆེད་པའི་སྐྱེས་ཚོགས་ཀྱིས།',
  9: 'ད་ལྟ་ཟླ་བ་གྲྭ་པ་རེ་རྣམས་རབ་ཏུ་འཁོང་བར་བྱེད།'},
 'url': {0: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_AB00651_0085_355598_to_358568.wav',
  1: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_AB00651_0054_218185_to_220328.wav',
  2: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_AB00651_

In [9]:
df.to_csv(f"stt_ab_from_yt.tsv", index=False, sep="\t")

In [10]:
from pathlib import Path

import pandas as pd
from fast_antx.core import transfer
import numpy as np


def extract_tsv_text(tsvFile, ColumnNumber):
    """extracts text from dataframe using column number
    Args:
        tsvFile (Dataframe): dataframe of predicted tsv file
        ColumnNumber (integer/string):column name of the text to be extracted

    Returns:
        string: extracted text from tsv file
    """
    # read the tsv file
    predictedText = tsvFile[ColumnNumber].tolist()
    # to avoid unwanted splits in a word we replace space with _
    for count, text in enumerate(predictedText):
        predictedText[count] = predictedText[count].replace(" ", "_")
    predictedText = "\n".join(" ".join(predictedText).split())
    print("extracted text from tsv file..")
    return predictedText


def get_original_text(OriginalText):
    """reads the original text and removes unwanted characters

    Args:
        OriginalText (string): location of the original text file

    Returns:
        string: original text without unwanted characters
    """
    target = Path(f"{OriginalText}").read_text(encoding="utf-8")
    # remove unwanted characters
    target = target.replace("“", "").replace("”", "")
    print("extracted text from original file..")

    return target


def transfer_text(OriginalText, PredictedTSV, file_name, ColumnNumber='inference_transcript'):
    """transfers the annotation from predicted text to original text and returns a dataFrame

    Args:
        OriginalText (string): location of the original string
        PredictedTSV (string): location of the predicted tsv file
        ColumnNumber (int/string): name of the column in which transcribed text is there in .tsv file

    Returns:
        dataFrame: dataFrame that contains transferred annotation on original text
    """
    tsvFile = pd.read_csv(f"{PredictedTSV}", sep="\t")
    tsvFile = tsvFile[tsvFile['file_name'].str[0:11] == file_name]

    tsvFile.sort_values(by=['file_name'], inplace=True)

    source = extract_tsv_text(tsvFile, ColumnNumber)
    target = get_original_text(OriginalText)
    annotation = [["segment", "(\n)"]]
    transferredText = transfer(source, annotation, target).split("\n")
    if len(transferredText) > len(tsvFile):
        transferredText = transferredText[:len(tsvFile)]
        tsvFile[ColumnNumber] = transferredText
        status= f'Truncated {abs(len(transferredText)-len(tsvFile))}'
    elif len(transferredText) < len(tsvFile):
        transferredText = transferredText + [np.nan]*(len(tsvFile) - len(transferredText))
        tsvFile[ColumnNumber] = transferredText
        status=f'Padded {abs(len(transferredText)-len(tsvFile))}'
    else:
        tsvFile[ColumnNumber] = transferredText
        status='Normal'

    # returns a dataFrame
    return tsvFile, status

In [13]:
temp = []
for file_name in [f"STT_AB00{x}" for x in range(645,692)]:
    transfer_text_df, status = transfer_text(f'etexts/{file_name}.txt',f'stt_ab_from_yt.tsv', file_name)
    temp.append(transfer_text_df)
    print(status)
df = pd.concat(temp)



extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extracted text from original file..
Normal
extracted text from tsv file..
extract

In [14]:
df.head()

,file_name,url,inference_transcript,audio_duration
1196,STT_AB00645_0001_3535_to_6623,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,01 སྡེ་སྲིད་སངས་རྒྱས་རྒྱ་མཚོ། སྡེ་སྲིད་སངས་རྒ...,3.088
2646,STT_AB00645_0002_32003_to_34500,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,༄༅། ། སྡེ་སྲིད་སངས་རྒྱས་རྒྱ་མཚོ་ནི། ཕྱི་ལོ་...,2.497
151,STT_AB00645_0003_43900_to_48878,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,འཁྲུངས། སྐུ་ན་གཞོན་དུས་ནས་ལྷན་སྐྱེས་སུ་རྣམ་རི...,4.978
2359,STT_AB00645_0004_49604_to_54020,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,དགུང་ལོ་བརྒྱད་ཡས་མས་ནས་བཟུང་༸རྒྱལ་དབང་ལྔ་པ་ཆ...,4.416
1411,STT_AB00645_0005_49604_to_56985,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྡེ་སྲིད་འཕྲིན་ལས་རྒྱ་མཚོ། འདར་ལོ་ཙཱ་བ,7.381


In [15]:
group_ab_ga_id = 1
group_ab_gb_id = 2
group_ab_gc_id = 7

In [16]:
df['group_id'] = group_ab_ga_id

In [17]:
df['state'] = 'imported'
df.fillna('', inplace=True)
def filter_length(st):
    return len(st) < 500

In [18]:
df = df[df['inference_transcript'].apply(lambda x: len(x) < 500)]

In [21]:
last_db_id = get_max_db_id()

id_arr = list(range(last_db_id + 1, df.shape[0] + last_db_id + 1))

df['id'] = id_arr


The maximum ID in the 'Task' table is: 448380


In [22]:
df.head()

,file_name,url,inference_transcript,audio_duration,group_id,state,id
1196,STT_AB00645_0001_3535_to_6623,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,01 སྡེ་སྲིད་སངས་རྒྱས་རྒྱ་མཚོ། སྡེ་སྲིད་སངས་རྒ...,3.088,1,imported,448381
2646,STT_AB00645_0002_32003_to_34500,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,༄༅། ། སྡེ་སྲིད་སངས་རྒྱས་རྒྱ་མཚོ་ནི། ཕྱི་ལོ་...,2.497,1,imported,448382
151,STT_AB00645_0003_43900_to_48878,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,འཁྲུངས། སྐུ་ན་གཞོན་དུས་ནས་ལྷན་སྐྱེས་སུ་རྣམ་རི...,4.978,1,imported,448383
2359,STT_AB00645_0004_49604_to_54020,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,དགུང་ལོ་བརྒྱད་ཡས་མས་ནས་བཟུང་༸རྒྱལ་དབང་ལྔ་པ་ཆ...,4.416,1,imported,448384
1411,STT_AB00645_0005_49604_to_56985,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྡེ་སྲིད་འཕྲིན་ལས་རྒྱ་མཚོ། འདར་ལོ་ཙཱ་བ,7.381,1,imported,448385


In [23]:
df.to_csv('stt_ab_upload_new.csv', index=False)

In [24]:
df.shape

(2662, 7)